Определите функцию is_satisfiable(присваивание, предложения), которая принимает логическое присваивание (список истинностных значений) и набор предложений (список списков, где каждый внутренний список содержит 3 целых числа, представляющих литералы). Эта функция возвращает True, если присваивание удовлетворяет всем предложениям, и False в противном случае.

Определите функцию oracle(предложения, num_variables), которая принимает набор предложений и количество логических переменных и возвращает квантовый оракул, который распознает удовлетворяющие присваивания.

Определите основную функцию 3-SAT-решателя grover_3sat_solver(предложения, num_variables), которая принимает набор предложений и количество логических переменных.

a. Создайте квантовый оракул, используя функцию oracle(предложения, num_variables).

б. Запустите алгоритм Гровера, используя квантовый оракул.

c. Если найдено удовлетворяющее заданию, верните его. В противном случае верните None.

In [ ]:
import itertools
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, transpile, assemble
from qiskit.quantum_info import Statevector
from qiskit.aqua.algorithms import Grover
from qiskit.aqua.components.oracles import CustomCircuitOracle

def is_satisfiable(assignment, clauses):
    for clause in clauses:
        if not any(assignment[abs(lit) - 1] if lit > 0 else not assignment[abs(lit) - 1] for lit in clause):
            return False
    return True

def oracle(clauses, num_variables):
    def oracle_function(assignment):
        return is_satisfiable(assignment, clauses)

    return oracle_function

def grover_3sat_solver(clauses, num_variables):
    oracle_function = oracle(clauses, num_variables)
    oracle_instance = CustomCircuitOracle(variable_register=QuantumRegister(num_variables, name='v'),
                                         output_register=ClassicalRegister(1, name='o'),
                                         circuit=oracle_function,
                                         evaluate_classically_callback=oracle_function)

    grover_instance = Grover(oracle=oracle_instance)

    result = grover_instance.run(quantum_instance=Aer.get_backend('statevector_simulator'))

    if 'result' in result:
        return result['result']
    else:
        return None

# Example usage:
clauses = [
    [1, 2, -3],
    [-1, -2, 3],
    [-2, 3, 4],
    [-3, -4, 1]
]

num_variables = 4

satisfying_assignment = grover_3sat_solver(clauses, num_variables)
print("Satisfying assignment:", satisfying_assignment)
